In [17]:
import numpy as np
from gplearn.functions import _Function


def build_callable_program(program):
    # Check for single-node programs
    node = program[0]
    if isinstance(node, float):
        return lambda X: np.repeat(node, X.shape[0])
    if isinstance(node, int):
        return lambda X: X[:, node]

    apply_stack = []

    for node in program:
        if isinstance(node, _Function):
            apply_stack.append([node])
        else:
            # Lazily evaluate later
            apply_stack[-1].append(node)
        
        while len(apply_stack[-1]) == apply_stack[-1][0].arity + 1:
            # Apply functions that have sufficient arguments
            function = apply_stack[-1][0]
            terminals = apply_stack[-1][1:]
            intermediate_function = lambda X, f=function, t=terminals: f(
                *[translate_terminal(terminal)(X) for terminal in t])
            if len(apply_stack) != 1:
                apply_stack.pop()
                apply_stack[-1].append(intermediate_function)
            else:
                return intermediate_function
            
def translate_terminal(terminal):
    if isinstance(terminal, int):
        return lambda X: X[:, terminal]
    if isinstance(terminal, float):
        return lambda X: np.repeat(terminal, X.shape[0])
    if callable(terminal):
        return terminal
    

In [19]:
from gplearn.functions import add2, mul2, neg1

program = [add2, mul2, 0.5, 0, neg1, 1] # (0.5 * x1) + (-x2)

func = build_callable_program(program)

x = np.array([[1, 2], [3, 4]])
func(x)

array([-1.5, -2.5])